In [ ]:
import boto3
from PIL import Image
import os
from botocore.exceptions import ClientError


def test_process_image():
    # Set up the test data
    s3 = boto3.client(
        "s3",
        endpoint_url="https://13583f5ff84f5693a4a859a769743849.r2.cloudflarestorage.com",
        aws_access_key_id="52733bb777295dbf8912df8ae9549466",
        aws_secret_access_key="8bfc25e62071097ae93aed260702353d9341a80ee352e68561f9ac95e983055f",
        region_name="auto",
    )
    bucket = s3.Bucket("cutoutimagestore")
    for obj in bucket.objects.all():
        print(obj.key)
    images = ["Screenshot 2023-06-25 145608.png", "Screenshot 2023-06-25 151400.png"]
    # for image in images:
    #     with open(image, 'w') as f:
    #         f.write('test')
    #     bucket.upload_file(image, image)
    s3.Bucket("cutoutimagestore").upload_file("image1.jpg", "image1.jpg")
    # Call the function being tested
    for i, filename in enumerate(images):
        print("Filename is:", filename)
        s3.Bucket("cutoutimagestore").download_file(filename, filename)
        # list_files(".")
        image = Image.open(filename)
        print(image)
        # process_image(image, filename, 'test', 'test', 'test')

    # Clean up the test data
    # for image in images:
    #     os.remove(image)
    #     bucket.Object(image).delete()


def generate_preview_url(bucket_name, key):
    s3 = boto3.client(
        "s3",
        endpoint_url="https://13583f5ff84f5693a4a859a769743849.r2.cloudflarestorage.com",
        aws_access_key_id="52733bb777295dbf8912df8ae9549466",
        aws_secret_access_key="8bfc25e62071097ae93aed260702353d9341a80ee352e68561f9ac95e983055f",
        region_name="auto",
    )

    try:
        response = s3.generate_presigned_url(
            "get_object",
            Params={
                "Bucket": bucket_name,
                "Key": key,
                "ResponseContentType": "image/jpeg",
            },
        )
    except ClientError as e:
        print(e)
        return None

    return response


img = Image.open("image1.jpg")
width, height = img.size
url = generate_preview_url(
    "cutoutimagestore", "image1.jpg"
)
print(url)

# test_process_image()

In [ ]:
import os
from io import BytesIO
from PIL import Image
from s3FileHandler import Boto3Client

# Initialize a Boto3Client object
client = Boto3Client()

# Download a file from S3
# client.download_from_s3("cutoutimagestore", "downloads")

# Upload a file to S3
# with open("image1.jpg", "rb") as f:
#     file_body = BytesIO(f.read())
client.upload_to_s3("cutoutimagestore", "image1.jpg", "test/image1.jpg")

# Generate a presigned URL for a file in S3
url = client.generate_presigned_url("cutoutimagestore", "image1.jpg")
print(url)

# Cutout handler test

In [1]:
import os
from PIL import Image
from cutoutHandler import CutoutHandler
from segment_anything import sam_model_registry, SamPredictor
from ultralytics import YOLO

# LOAD SEGMENT ANYTHING MODEL
sam_checkpoint = "sam_vit_b_01ec64.pth"
model_type = "vit_b"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)

# LOAD YOLO MODEL
model = YOLO("yolov8n.pt")

# Define the path to the image file
image_path = "00100sPORTRAIT_00100_BURST20221002203407257_COVER.jpg"

# Load the image using PIL
image = Image.open(image_path)

# Define the prompt for the cutout
prompt = "person"

# Create a CutoutHandler object
handler = CutoutHandler(model, predictor)

# Process the image and generate the cutouts
cutouts = handler.process_image(image, os.path.basename(image_path), prompt)

# print(cutouts)
# Display the generated cutouts
for cutout in cutouts:
    cutout_image = Image.open(cutout)
    cutout_image.show()

2023-07-20 00:05:55,564 [DEBUG] Processing image '00100sPORTRAIT_00100_BURST20221002203407257_COVER.jpg' with prompt 'person'


array_shape:  (1728, 2304, 3)



0: 480x640 1 vase, 77.8ms
Speed: 11.6ms preprocess, 77.8ms inference, 26.8ms postprocess per image at shape (1, 3, 480, 640)
2023-07-20 00:06:00,254 [DEBUG] Processed 0 masks for prompt 'person' in image '00100sPORTRAIT_00100_BURST20221002203407257_COVER.jpg'


: 